In [26]:
# Change log:
# Добавлена очистка
# Добавлена обработка финального результата

def clearing(data):
    import re
    import ast
    import pandas as pd
    import numpy as np
    
    top = ['WA', 'NH', 'MN', 'UT', 'VT', 'MD', 'VA', 'MA', 'NE', 'CO', 'WI', 'NJ', 'FL', 'IA', 'ND', 'ID']
    mid = ['GA', 'NC', 'CA', 'SD', 'CT', 'KS', 'DE', 'HI', 'NY', 'RI', 'OR', 'MO', 'MT', 'TN', 'WY', 'ME']
    
    data = data.drop(['MlsId', 'stories', 'mls-id', 'private pool', 'city', 'street', 'zipcode', 'propertyType', 'status'],axis = 1)
    def state(x):
        if x in top:
            return 2
        if x in mid:
            return 1
        return 0
    
    def func_cool(x):
        if x == '':
            return 0
        return 1

    def func_pool(x):
        if type(x) != int:
            x = x.replace(x, '1')
        return int(x)
    
    def func_stor(x):
        if type(x) != int:
            x = re.findall(r'\d+',x)
            if (len(x) > 1):
                x = x[0]
            elif len(x) == 1:
                x = x[0]
            if not x:
                return 1
        return x
    
    def func_cost(x):
        if type(x) != int:
            x = re.findall(r'\d+',x)
            if len(x) > 2:
                x = x[0] + x[1] + x[2]
            elif len(x) > 1:
                x = x[0] + x[1]
            elif len(x) == 1:
                x = x[0]
            if not x:
                return 510000
        return int(x)
    
    def func_area(x):
        if type(x) != int:
            x = re.findall(r'\d+',x)
            if len(x) > 1:
                x = x[0] + x[1]
            elif len(x) == 1:
                x = x[0]
            if not x:
                return 2235
        return int(x)

    def func_lot(x):
        if type(x) != int:
            x = re.findall(r'\d+',x)
            if len(x) > 1:
                x = x[0] + x[1]
            elif len(x) == 1:
                x = x[0]
            if not x:
                return 4400
        return int(x)
    
    def func_price(x):
        if type(x) != int:
            x = re.findall(r'\d+',x)
            if len(x) > 1:
                x = x[0] + x[1]
            elif len(x) == 1:
                x = x[0]
            if not x:
                return 220
        return int(x)
    
    def func_built(x):
        x = re.findall(r'\d+',x)
        if len(x) > 1:
            x = x[0] + x[1]
        elif len(x) == 1:
            x = x[0]
        if (not x) | (x == 'None'):
            return 1977
        return x
        
    def data_func1(x):
        a = ast.literal_eval(x)
        return a['atAGlanceFacts'][0].get('factValue')

    def data_func2(x):
        a = ast.literal_eval(x)
        return a['atAGlanceFacts'][1].get('factValue')

    def data_func4(x):
        a = ast.literal_eval(x)
        return a['atAGlanceFacts'][3].get('factValue')

    def data_func6(x):
        a = ast.literal_eval(x)
        return a['atAGlanceFacts'][5].get('factValue')

    def data_func7(x):
        a = ast.literal_eval(x)
        return a['atAGlanceFacts'][6].get('factValue')
    
    def data_func8(x):
        m = []
        a = ast.literal_eval(x)
        a = dict(a[0])
        k = a['data'].get('Distance')
        for i in k:
            h = re.findall(r'\d+',i)
            if len(h) == 0:
                continue
            if len(h) == 1:
                m.append(float(h[0]))
                continue
            d = h[0] + '.' + h[1]
            m.append(float(d))
        if len(m) == 0:
            return 0
        else:
            return min(m)
        return int(x)
    
    def func_rebuilt(row):
        if row['rebuilt'] == 0:
            return row['built']
        if row['rebuilt'] == '':
            return row['built']
        return row['rebuilt']
    
    data['PrivatePool'] = data['PrivatePool'].fillna(0).apply(func_pool)
    data['fireplace'] = data['fireplace'].fillna(0).apply(func_pool)
    data['baths'] = data['baths'].fillna(1).apply(func_stor)
    data['beds'] = data['beds'].fillna(1).apply(func_stor)
    data['built'] = data['homeFacts'].apply(data_func1)
    data['rebuilt'] = data['homeFacts'].apply(data_func2)
    data['cooling'] = data['homeFacts'].apply(data_func4)
    data['lotsize'] = data['homeFacts'].apply(data_func6)
    data['price'] = data['homeFacts'].apply(data_func7)
    data['distance'] = data['schools'].apply(data_func8).fillna(0)
    data = data.drop(['homeFacts', 'schools'], axis = 1)
    data['cooling'] = data['cooling'].apply(func_cool)
    data['state'] = data['state'].apply(state)
    data['sqft'] = data['sqft'].fillna(2235).apply(func_area)
    data['target'] = data['target'].fillna(510000).apply(func_cost)
    data['lotsize'] = data['lotsize'].fillna(4400).apply(func_lot)
    data['price'] = data['price'].fillna(220).apply(func_price)
    data['built'] = data['built'].fillna(1977)
    data['built'] = data['built'].astype(str)
    data['built'] = data['built'].apply(func_built)
    data['built'] = pd.to_numeric(data['built'])
    data['rebuilt'] = data['rebuilt'].fillna(0)
    data['rebuilt'] = data.apply(lambda row: func_rebuilt(row), axis=1)
    data['sqft'] = np.log1p(data['sqft'])
    data = data.drop(['built'], axis = 1)
    return data

In [27]:
import pickle
import pandas as pd
filename = 'data.csv'
data = pd.read_csv(filename)
with open('mymodel.pkl', 'rb') as pkl_file:
    regressor = pickle.load(pkl_file)
data = data[:1000]

In [28]:
cleared = clearing(data)
cleared.columns

Index(['baths', 'fireplace', 'sqft', 'beds', 'state', 'PrivatePool', 'target',
       'rebuilt', 'cooling', 'lotsize', 'price', 'distance'],
      dtype='object')

In [29]:
y_test = cleared['target']
cleared = cleared.drop('target', axis = 1)

In [30]:
pred = np.expm1(regressor.predict(cleared))

In [31]:
for x in range(len(pred)):
    pred[x] = round(pred[x]/1000)*1000

In [32]:
y_pred = pd.DataFrame(pred)

In [33]:
full = cleared.join(y_pred)

In [34]:
full.columns = ['baths', 'fireplace', 'sqft', 'beds', 'state', 'PrivatePool', 
       'rebuilt', 'cooling', 'lotsize', 'price', 'distance', 'target']

In [35]:
full

,baths,fireplace,sqft,beds,state,PrivatePool,rebuilt,cooling,lotsize,price,distance,target
0,3,1,7.972811,4,1,0,2019,0,4400,144,2.70,417000.0
1,3,0,7.574558,3,2,0,2019,0,5828,159,1.01,309000.0
2,2,1,8.006701,3,1,1,1967,1,8626,965,1.19,2926000.0
3,8,1,8.773075,5,0,0,2006,1,8220,371,0.10,2387000.0
4,1,0,7.712444,1,2,0,1977,0,10019,220,3.03,484000.0
...,...,...,...,...,...,...,...,...,...,...,...,...
995,2,1,7.768533,4,0,0,1900,1,6882480,317,4.40,752000.0
996,2,1,7.678326,4,2,0,1981,0,66646,282,1.10,603000.0
997,5,0,8.268475,6,2,0,1986,1,41,205,0.70,811000.0
998,1,0,7.161622,1,0,0,1972,1,5998,39,1.00,51000.0
